# TAKE A LOOK AT THIS: https://www.kaggle.com/code/ohmeow/iterate-like-a-grandmaster-blurrified

In [1]:
dependencies = [
    "kagtool",
    "kaggle",
    "fastai",
    "nbdev",
    "datasets"
]

!pip install -U {" ".join(dependencies)}

from fastai.imports import *


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from fastai.text.all import *

In [3]:
from kagtool.datasets.kaggle_downloader import KaggleDownloader

dataset_name = 'us-patent-phrase-to-phrase-matching'
creds = '{"username":"slashafk","key":"2d0613a35898cc347c1607c64dd143ac"}'

dataset_path = KaggleDownloader(dataset_name, creds).load_or_fetch_kaggle_dataset()
df = pd.read_csv(dataset_path/'train.csv')
df

us-patent-phrase-to-phrase-matching.zip: Skipping, found more recently modified local copy (use --force to force download)


,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [4]:
# import os
# import random
# import shutil

# !rm -rf ten ten_tok

# def copy_files(source_dir, dest_dir, percentage):
#     # Get a list of all files in the source directory
#     files = os.listdir(source_dir)
    
#     # Calculate the number of files to copy based on the percentage
#     num_files = int(len(files) * percentage)
    
#     # Randomly select the files to copy
#     files_to_copy = random.sample(files, num_files)
    
#     # Copy the selected files to the destination directory
#     for file in files_to_copy:
#         shutil.copy(os.path.join(source_dir, file), dest_dir)


# source_directory = 'lesspatents'
# destination_directory = 'ten'
# !mkdir -p {destination_directory}
# percentage = 0.2

# copy_files(source_directory, destination_directory, percentage)

# !du -sh {destination_directory}

In [5]:
path = Path('/workspace/10')
path

Path('/workspace/10')

In [6]:
!du -sh {path}

741M	/workspace/10


In [7]:
get_text_files(path)

(#78503) [Path('/workspace/10/smaller_file_535330.txt'),Path('/workspace/10/smaller_file_752307.txt'),Path('/workspace/10/smaller_file_157024.txt'),Path('/workspace/10/smaller_file_682324.txt'),Path('/workspace/10/smaller_file_137908.txt'),Path('/workspace/10/smaller_file_205902.txt'),Path('/workspace/10/smaller_file_421437.txt'),Path('/workspace/10/smaller_file_525123.txt'),Path('/workspace/10/smaller_file_95153.txt'),Path('/workspace/10/smaller_file_101262.txt')...]

In [8]:
# TextBlock(tok_tfm=tkz, is_lm=True, min_freq=3, max_vocab=60000)
text_block = TextBlock.from_folder(path, is_lm=True)

In [9]:
block = DataBlock(
    blocks=text_block,
    get_items=get_text_files, splitter=RandomSplitter(0.1)
)

# get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

# dls_lm = DataBlock(
#     blocks=TextBlock.from_folder(path, is_lm=True),
#     get_items=get_imdb, splitter=RandomSplitter(0.1)
# ).dataloaders(path, path=path, bs=128, seq_len=80)

In [10]:
# block.summary(path, bs=128, seq_len=80)

In [11]:
dls_lm = block.dataloaders(path, path=path, bs=128, seq_len=80, num_workers=8)
dls_lm.show_batch(max_n=3)

,text,text_
0,"xxbos xxmaj method xxmaj for xxmaj manufacturing xxmaj touch xxmaj substrate , xxmaj touch xxmaj substrate xxmaj and xxmaj touch xxmaj display xxmaj screen \n xxmaj the present disclosure relates to a method for manufacturing a touch substrate . xxmaj the method includes forming a surface strengthening layer on a surface of a touch mother substrate ; cutting the touch mother substrate formed with the surface strengthening layer to form a plurality of touch sub - substrates ; and removing","xxmaj method xxmaj for xxmaj manufacturing xxmaj touch xxmaj substrate , xxmaj touch xxmaj substrate xxmaj and xxmaj touch xxmaj display xxmaj screen \n xxmaj the present disclosure relates to a method for manufacturing a touch substrate . xxmaj the method includes forming a surface strengthening layer on a surface of a touch mother substrate ; cutting the touch mother substrate formed with the surface strengthening layer to form a plurality of touch sub - substrates ; and removing the"
1,"cyclic ethers , 1,4 - cyclic ethers , anhydrides , lactones , lactams , cyclic ethers , siloxanes , oxazolines , xxunk , and xxunk ; 20 . xxmaj the composition of claim 1 wherein the structural diluent is selected from the group consisting of 3,4 - xxunk - xxunk ( xxunk ) , xxmaj xxunk - xxunk ( xxunk ) , difunctional bisphenol a / epichlorohydrin derived liquid epoxy resin ( epon 828 ) , xxmaj bisphenol a epoxy","ethers , 1,4 - cyclic ethers , anhydrides , lactones , lactams , cyclic ethers , siloxanes , oxazolines , xxunk , and xxunk ; 20 . xxmaj the composition of claim 1 wherein the structural diluent is selected from the group consisting of 3,4 - xxunk - xxunk ( xxunk ) , xxmaj xxunk - xxunk ( xxunk ) , difunctional bisphenol a / epichlorohydrin derived liquid epoxy resin ( epon 828 ) , xxmaj bisphenol a epoxy resin"
2,"first level to decrease unintentional release of coins . xxmaj the lip or flange can have various dimensions , but some possible ones include : 60 mm radius over quarter apertures , 34 mm radius over xxunk apertures , 36 mm radius over dime apertures , and 48 mm radius over nickel apertures . xxmaj additional finger gaps are provided in the perimeter of the card to aid in release of coins in the first level , the gaps having","level to decrease unintentional release of coins . xxmaj the lip or flange can have various dimensions , but some possible ones include : 60 mm radius over quarter apertures , 34 mm radius over xxunk apertures , 36 mm radius over dime apertures , and 48 mm radius over nickel apertures . xxmaj additional finger gaps are provided in the perimeter of the card to aid in release of coins in the first level , the gaps having dimensions"


In [12]:
import gc
import torch

try:
    del learn
    gc.collect()
    torch.cuda.empty_cache()
except:
    pass

learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [13]:
def predict_next(txt, learn):
    N_WORDS = 40
    N_SENTENCES = 2
    preds = [learn.predict(txt, N_WORDS, temperature=0.75) 
             for _ in range(N_SENTENCES)]
    print("\n".join(preds))

predict_next("invention", learn)

invention ( 2010 ABC News Series ) = and 2011 TV series = : HBO Series USA , Inc . ( ABC ) , is a Canadian reality television series which
invention Pilot ( Glee ) = ( " Pilot " ) is the final episode of the fifth season of the American television series Glee . It was written by Sam Weiss and


In [14]:
# learn.lr_find()

In [15]:
learn.fit_one_cycle(20, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.700938,3.633403,0.363512,37.841385,08:34
1,3.440892,3.372326,0.383866,29.146246,08:34
2,3.267343,3.271772,0.391736,26.357996,08:33
3,3.197778,3.233498,0.394351,25.368237,08:34
4,3.145452,3.215041,0.395788,24.904318,08:33
5,3.123330,3.197077,0.397440,24.460915,08:34
6,3.094718,3.189754,0.397838,24.282448,08:33
7,3.088689,3.178291,0.399046,24.005695,08:31
8,3.048592,3.172411,0.399353,23.864958,08:37
9,3.065909,3.163194,0.400405,23.646000,08:34


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [16]:
learn.save_encoder('finetuned')

In [17]:
predict_next("invention", learn)

invention Method disclosed for a method of generating a coding scheme for a target , the method comprises selecting a method of identifying the target gene from the target gene using the target gene . The method of claim
invention The Radiation Sensing Device is a device for measuring radiation emitted from a patient 's skin of the patient , a patient 's body that is physically attached . The patient body is positioned within


# Classifier DataLoaders

In [18]:
def preprocess_inputs(df):
    sep = " [s] "
    df['inputs'] = df.context + sep + df.anchor + sep + df.target
    df['inputs'] = df.inputs.str.lower()
    df['section'] = df.context.str[0]
    df['sectok'] = '[' + df.section + ']'
    df['inputs'] = df.sectok + sep + df.context + sep + df.anchor.str.lower() + sep + df.target

preprocess_inputs(df)
df.head()

,id,anchor,target,context,score,inputs,section,sectok
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,[A] [s] A47 [s] abatement [s] abatement of pollution,A,[A]
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,[A] [s] A47 [s] abatement [s] act of abating,A,[A]
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,[A] [s] A47 [s] abatement [s] active catalyst,A,[A]
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,[A] [s] A47 [s] abatement [s] eliminating process,A,[A]
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,[A] [s] A47 [s] abatement [s] forest region,A,[A]


In [19]:
doc(TextBlock.from_df)

In [31]:
anchors = df.anchor.unique()
np.random.seed(42)
np.random.shuffle(anchors)
val_prop = 0.25
val_sz = int(len(anchors) * val_prop)
val_anchors = anchors[:val_sz]
is_val = np.isin(df.anchor, val_anchors)
idxs = np.arange(len(df))
val_idxs = idxs[ is_val]
trn_idxs = idxs[~is_val]



text_block = TextBlock.from_df(text_cols='inputs', vocab=dls_lm.vocab)

# for some reason this whole shit works only if the column name is called text?
# seems like text_block renames the column to 'text'
cr_x = ColReader('text')

# but apparently for Y we can keep the original name? using category will even fail!
# behavior is completely different between get_x and get_y
cr_y = ColReader('score')


dls_clas = DataBlock(
    blocks=(text_block, RegressionBlock),
    get_x=cr_x,
    get_y=cr_y,
    splitter=IndexSplitter(val_idxs)
).dataloaders(df, bs=128, seq_len=72, num_workers=8)

dls_clas.show_batch(max_n=3)

,text,text_
0,xxbos [ c ] [ s ] xxup xxunk [ s ] xxunk [ s ] 3 4 hydroxyphenyl 6 methoxy 7 xxunk 345 trihydroxy 6 xxunk 2 xxunk 4 one,0.75
1,xxbos [ c ] [ s ] xxup xxunk [ s ] rizatriptan [ s ] nn dimethyl 2 5 124 xxunk 1 ylmethyl 1h xxunk 3 xxunk,0.75
2,xxbos [ b ] [ s ] xxup b22 [ s ] xxunk [ s ] heating calcium oxide and aluminium oxide together at high temperatures,0.75


In [39]:
import gc
import torch

try:
    del classify
    gc.collect()
    torch.cuda.empty_cache()
except:
    pass

classify = text_classifier_learner(dls_clas, AWD_LSTM,
                                drop_mult=0.5, 
                                metrics=[PearsonCorrCoef()],
                                loss_func=MSELossFlat()
                               ).to_fp16()

classify = classify.load_encoder('/workspace/10/models/finetuned')
classify.fit_one_cycle(1, 2e-2)

text_to_classify = "[A] [s] A47 [s] abatement [s] eliminating process"
classify.predict(text_to_classify)

epoch,train_loss,valid_loss,pearsonr,time
0,0.079366,0.067052,0.092475,00:05


((0.36474609375,), tensor([0.3647]), tensor([0.3647]))

What's happening between these the above line and the below one actually? Isn't the second line making the first line useless??

But we need to pass the dataloaders for classification, so probably no? That's a bit strange

### Similar To Gradual Unfreezing?

In [24]:
# classify.fine_tune(10)

### In just one epoch we should get a pretty good result already

In [25]:
classify.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,pearsonr,time
0,0.067743,0.063648,0.268238,00:05


### Gradual Unfreezing

In [26]:
classify.freeze_to(-2)
classify.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

classify.freeze_to(-3)
classify.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

classify.unfreeze()
classify.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,pearsonr,time
0,0.064823,0.057813,0.379558,00:06


epoch,train_loss,valid_loss,pearsonr,time
0,0.059155,0.055590,0.445092,00:06


epoch,train_loss,valid_loss,pearsonr,time
0,0.055416,0.052373,0.480949,00:06
1,0.051349,0.051059,0.493839,00:06


In [27]:
# Now you can use it for inference
text_to_classify = "[A] [s] A47 [s] abatement [s] act of abating 	"
classify.predict(text_to_classify)

((0.297119140625,), tensor([0.2971]), tensor([0.2971]))

In [28]:
from fastai.metrics import PearsonCorrCoef
# AttributeError: Can't pickle local object 'PearsonCorrCoef.<locals>.pearsonr'
from scipy.stats import pearsonr

model_name = os.path.basename(path)
model_path = '/workspace/'+model_name+'.pkl'
classify.export(model_path)

AttributeError: Can't pickle local object 'PearsonCorrCoef.<locals>.pearsonr'

# US Patent

In [ ]:
from fastai.text.all import load_learner

# Load the exported learner
classifier = load_learner(model_path)

# Now you can use it for inference
text_to_classify = "[A] [s] A47 [s] abatement [s] forest region"
classifier.predict(text_to_classify)